In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import json
import pprint

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from matplotlib import gridspec
import pprint
from scipy.stats import gaussian_kde

detector_path = '../..'
results_path = os.path.join(detector_path, 'results')
sys.path.append(detector_path)

from sleep.utils import constants
from sleep.utils import checks
from sleep.data.mass_ss import MassSS
# from sleep.inta import INTA
from sleep.data import postprocessing
from sleep.data import data_ops
from sleep.data import metrics

DPI = 150

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
# Best ones:
# MASS: ckpt_folder = os.path.join('20190408_bsf_train_mass', 'bsf', 'avg'), thr=0.45
# INTA: ckpt_folder = os.path.join('20190407_bsf_train_inta', 'bsf', 'avg'), thr=0.475

# Select database
dataset_name = constants.MASS_SS_NAME
# Select predictions ckpt folder
ckpt_folder = os.path.join('20190408_bsf_train_mass', 'bsf', 'avg')
thr = 0.45
verbose = False

# Load data

# Marks are binary sequences for each page, 200 fs resolution
# Load data
checks.check_valid_value(
    dataset_name, 'dataset_name',
    [constants.MASS_SS_NAME, constants.INTA_SS_NAME, constants.MASS_KC_NAME])
if dataset_name == constants.MASS_SS_NAME:
    dataset = MassSS(load_checkpoint=True)
elif dataset_name == constants.INTA_SS_NAME:
    dataset = IntaSS(load_checkpoint=True)
else:
    dataset = MassKC(load_checkpoint=True)

# Get training set ids
print('Loading train set and splitting train/val')
all_train_ids = dataset.train_ids
print('All Training set IDs:', all_train_ids)
# Get test data
print('Loading test set')
test_ids = dataset.test_ids
print('Testing set IDs:', test_ids)
idx_dict = {'alltrain': all_train_ids, 'test': test_ids}

# Get subjects data, with the expert used for training
print('Loading signals and marks')
# set_list = ['train', 'val', 'test']
set_list = ['alltrain', 'test']
x = {}
y = {}
pages = {}
x['alltrain'], y['alltrain'] = dataset.get_subset_data(all_train_ids, which_expert=1, verbose=verbose)
x['test'], y['test'] = dataset.get_subset_data(test_ids, which_expert=1, verbose=verbose)
print('Loading pages')
pages['alltrain'] = dataset.get_subset_pages(all_train_ids, verbose=verbose)
pages['test'] = dataset.get_subset_pages(test_ids, verbose=verbose)

# Load predictions (probability vectors for each page), 200/factor resolution (default factor 8)
ckpt_path = os.path.abspath(os.path.join(results_path, 'predictions_%s' % dataset_name, ckpt_folder))
print('Loading predictions from %s' % ckpt_path)
y_pred = {}
for set_name in set_list:
    y_pred[set_name] = np.load(os.path.join(ckpt_path, 'y_pred_%s.npy' % set_name), allow_pickle=True)
    # Keep only class 1 probability
    y_pred[set_name] = [this_y_pred[..., 1] for this_y_pred in y_pred[set_name]]
print('Done')

Dataset mass_ss with 15 patients.
Train size: 11. Test size: 4
Train subjects: 
 [1, 3, 5, 7, 9, 10, 11, 14, 17, 18, 19]
Test subjects: 
 [2, 6, 12, 13]
Loading from checkpoint... Loaded
Loading train set and splitting train/val
All Training set IDs: [1, 3, 5, 7, 9, 10, 11, 14, 17, 18, 19]
Loading test set
Testing set IDs: [2, 6, 12, 13]
Loading signals and marks
Loading pages
Loading predictions from /home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190408_bsf_train_mass/bsf/avg


FileNotFoundError: [Errno 2] No such file or directory: '/home/ntapia/Projects/GitNico/sleep-rnn/results/predictions_mass_ss/20190408_bsf_train_mass/bsf/avg/y_pred_alltrain.npy'

In [60]:
subset_name = 'test'
subject_id = 13

# -----
idx_subject = idx_dict[subset_name].index(subject_id)
this_signal = x[subset_name][idx_subject]
this_marks_1 = y[subset_name][idx_subject]
this_n2_pages = pages[subset_name][idx_subject]
this_prob = y_pred[subset_name][idx_subject]

y_pred_thr = postprocessing.generate_mark_intervals(
    y_pred[subset_name][idx_subject], pages[subset_name][idx_subject], 200//8, 200, thr=thr)
y_pred_thr_seq = data_ops.inter2seq(y_pred_thr, 0, (pages[subset_name][idx_subject].max()+1)*200*20-1)
# Now reshape into pages
this_det = data_ops.extract_pages(y_pred_thr_seq, pages[subset_name][idx_subject], 200*20, border_size=0)

if dataset_name == constants.MASS_NAME:
    _, this_marks_2 = dataset.get_subject_data(subject_id, which_expert=2, verbose=True)
    channel_name = 'C3-CLE'
else:
    channel_name = 'F4-C4'
    
# make a color map of fixed colors
cmap = colors.ListedColormap(['white', '#c62828'])

def plot_page(page_idx):
    if dataset_name == constants.MASS_NAME:
        fig = plt.figure(figsize=(15, 4), dpi=DPI)
        gs = gridspec.GridSpec(3, 1, height_ratios=[4, 1, 1])
    else:
        fig = plt.figure(figsize=(15, 3), dpi=DPI)
        gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    
    page_idx = page_idx - 1
    
    segment_signal = this_signal[page_idx, :]
    segment_marks_1 = this_marks_1[page_idx, :]
    segment_prob = this_prob[page_idx, :]
    segment_det = this_det[page_idx, :]
    time_axis = np.arange(this_signal.shape[1]) / dataset.fs
    
    if dataset_name == constants.MASS_NAME:
        segment_marks_2 = this_marks_2[page_idx, :]
    
    gs_idx = 0
    
    # Signal
    y_max = 10
    ax = fig.add_subplot(gs[gs_idx])
    gs_idx = gs_idx + 1
    ax.plot(time_axis, segment_signal, linewidth=1, color='#455a64', label='EEG %s' % channel_name)
    ax.fill_between(time_axis, y_max * segment_marks_1, -y_max * segment_marks_1, facecolor='#c62828', alpha=0.3, label='Sleep Spindle')
    ax.set_yticks([])
    ax.set_xlim([0, 20])
    ax.set_ylim([-y_max, y_max])
    ax.set_title('Subject %d (%s-%s). Page in record: %d (intervals of 0.5s are shown).' 
                 % (subject_id, dataset_name.upper(), subset_name.capitalize(), this_n2_pages[page_idx]), fontsize=10)
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xticks(np.arange(0, 20, 0.5), minor=True)
    ax.grid(b=True, axis='x', which='minor')
    lg = ax.legend(loc='upper right', fontsize=8.5)
    # lh = lg.legendHandles[1]
    # lh.set_facecolor(lh.get_facecolor())
    # lh.set_alpha(1.0)
    ax.tick_params(labelsize=8.5)
    
    # Neural net
    delta_y = 0.1
    time_axis_short = time_axis[::8]
    ax = fig.add_subplot(gs[gs_idx])
    gs_idx = gs_idx + 1
    ax.plot(time_axis_short, segment_prob, color='#c62828', linewidth=1.5, zorder=7)
    ax.fill_between(time_axis, (1+delta_y)*segment_det, (-delta_y)*segment_det, facecolor='grey', zorder=6)
    ax.set_xticks([])
    ax.set_ylim([-delta_y, 1+delta_y])
    ax.set_xlim([0, 20])
    ax.set_yticks([0, 1])
    ax.set_yticks([thr], minor=True)
    ax.grid(b=True, axis='y', linewidth=1.5, which='minor', zorder=5)
    ax.tick_params(labelsize=8.5)
    ax.set_title('Model probability prediction (%1.2f threshold and postprocessed detections are shown)' % thr, fontsize=10)
    
    if dataset_name == constants.MASS_NAME:
        ax = fig.add_subplot(gs[gs_idx])
        ax.imshow(segment_marks_2[np.newaxis, :], interpolation=None, aspect='auto', cmap=cmap)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title('Second expert, not used for training (red indicates event)', fontsize=10)
    
    plt.tight_layout()
    plt.show()

Getting ID 13, 818 N2 pages, Expert 2


In [61]:
widgets.interact(
    lambda page_idx: plot_page(page_idx),
    page_idx=widgets.IntSlider(min=1,max=this_n2_pages.shape[0],step=1,value=1, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='page_idx', max=818, min=1), Out…